<a href="https://colab.research.google.com/github/kumar6rishabh/RCNN-from-scratch/blob/main/full_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets download -d tannergi/chess-piece-detection

 94% 156M/166M [00:01<00:00, 146MB/s]
100% 166M/166M [00:01<00:00, 152MB/s]


In [ ]:
!unzip *.zip

In [ ]:
import os
import numpy as np
import pandas as pd
import bs4
import lxml
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import pickle
import gc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm

from bs4 import BeautifulSoup as bs

import torch
import torch.nn as nn
from torchvision import models

In [ ]:
imgs = os.listdir("/content/Chess Detection/images")

In [ ]:
annos = sorted(os.listdir("/content/Chess Detection/annotations"))

In [ ]:
df = None

In [ ]:
train_img_inds , val_img_inds = train_test_split(range(len(imgs)) , test_size = 0.1 , random_state = 1234)

In [ ]:
def objects(f , df):
    fln = f.split(".")[0]
    f = open("/content/Chess Detection/annotations/" + f , "r")
    lines = f.readlines()
    content = " ".join(lines)
    bs_content = bs(content , "lxml")
    lis = bs_content.find_all("object")
    names_lis = bs_content.find_all("name")
    xmin_lis = bs_content.find_all("xmin")
    xmax_lis = bs_content.find_all("xmax")
    ymin_lis = bs_content.find_all("ymin")
    ymax_lis = bs_content.find_all("ymax")
    names = list(map(lambda x : x.text , names_lis))
    xmin = list(map(lambda x : int(x.text) , xmin_lis))
    xmax = list(map(lambda x : int(x.text) , xmax_lis))
    ymin = list(map(lambda x : int(x.text) , ymin_lis))
    ymax = list(map(lambda x : int(x.text) , ymax_lis))
    data = {"fln" : fln , "name" : names , "xmin" : xmin , "xmax" : xmax , "ymin" : ymin , "ymax" : ymax}
    if df is None:
        df = pd.DataFrame(data)
    else:
        tp = pd.DataFrame(data)
        df = pd.concat([df , tp] , ignore_index = True , axis = 0)
    return df

In [ ]:
for ann in annos:
    df = objects(ann , df)

In [ ]:
df.head()

,fln,name,xmin,xmax,ymin,ymax
0,IMG_1989,white-rook,1780,2319,573,1442
1,IMG_1990,white-rook,1880,2383,1052,1532
2,IMG_1991,white-rook,1325,1564,1112,1378
3,IMG_1992,white-knight,1613,2001,342,1098
4,IMG_1993,white-knight,1795,2165,880,1280


In [ ]:
def get_iou(rec , truths):
  x11 , y11 , x12 , y12 = rec[0] , rec[1] , rec[0] + rec[2] , rec[1] + rec[3]
  x21 , y21 , x22 , y22 = truths[0] , truths[1] , truths[2] , truths[3]
  x1 , y1 , x2 , y2 = max(x11 , x21) , max(y11 , y21) , min(x12 , x22) , min(y12 , y22)
  intersection = max(0 , x2 - x1) * max(0 , y2 - y1)
  area1 = (x12 - x11) * (y12 - y11)
  area2 = (x22 - x21) * (y22 - y21)
  union = area1 + area2 - intersection
  return intersection / union

In [ ]:
img_to_prop = {}

In [ ]:
for i , ig in enumerate(imgs):
  all_white = []
  all_black = []
  all_neg = []
  img = cv2.imread("/content/Chess Detection/images/" + ig)
  img = cv2.resize(img , (500 , 281))
  subset_df = df[df.fln == ig.split(".")[0]]
  ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
  ss.setBaseImage(img)
  ss.switchToSelectiveSearchQuality()
  rects = ss.process()
  for j , rec in enumerate(rects):
    if len(all_white) == 100 or len(all_black) == 100:
      break
    ious = []
    for t in range(subset_df.shape[0]):
      _ , name , xmin , xmax , ymin , ymax = subset_df.iloc[t]
      truths = [int(xmin / 8) , int(ymin / 8) , int(xmax / 8) , int(ymax / 8)]
      iou = get_iou(rec , truths)
      ious.append([iou , truths , name])
    for one_iou in ious:
      iou , truths , name = one_iou
      if iou > 0.7 and "white" in name:
        all_white.append([rec , truths])
        break
      elif iou > 0.7 and "black" in name:
        all_black.append([rec , truths])
        break
    for one_iou in ious:
      iou , truths , name = one_iou
      if iou > 0.3:
        break
    all_neg.append([rec , truths])
  img_to_prop[ig] = [all_white , all_black , all_neg]
  print(i , " done")

In [ ]:
ff = open("data.pkl" , "wb")
pickle.dump(img_to_prop , ff)
ff.close()

In [ ]:
a_file = open("/content/drive/MyDrive/pytorch/practise/rcnn/data.pkl", "rb")
img_to_prop = pickle.load(a_file)

In [ ]:
model = models.resnet18(pretrained = True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
tot_fr = model.fc.in_features

In [ ]:
class Identity(nn.Module):
  def __init__(self):
    super(Identity , self).__init__()
    
  def forward(self , x):
    return x

In [ ]:
class C_Layer(nn.Module):
  def __init__(self , tot_fr):
    super(C_Layer , self).__init__()
    self.layer1 = nn.Linear(tot_fr , 1)
    self.sig = nn.Sigmoid()
  def forward(self , x):
    x = self.layer1(x)
    x = self.sig(x)
    return x

In [ ]:
class R_Layer(nn.Module):
  def __init__(self , tot_fr):
    super(R_Layer , self).__init__()
    self.layer1 = nn.Linear(tot_fr , 4)
  def forward(self , x):
    x = self.layer1(x)
    return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
model.fc = Identity()
c_lay = C_Layer(tot_fr)
r_lay = R_Layer(tot_fr)

In [ ]:
checkpoints = torch.load("/content/drive/MyDrive/pytorch/practise/rcnn/five_epoch.pth.tar")
model.load_state_dict(checkpoints["model_state"])

<All keys matched successfully>

In [ ]:
model.to(device)
c_lay.to(device)
r_lay.to(device)

R_Layer(
  (layer1): Linear(in_features=512, out_features=4, bias=True)
)

In [ ]:
loss_fn_cls = nn.BCELoss()
loss_fn_reg = nn.MSELoss()
optimizer1 = torch.optim.SGD(model.parameters() , lr = 0.001)
optimizer2 = torch.optim.SGD(c_lay.parameters() , lr = 0.001)
optimizer3 = torch.optim.SGD(r_lay.parameters() , lr = 0.001)

In [ ]:
num_epochs = 1
model.train()
all_losses = []
for epoch in range(num_epochs):
  epoch_loss = 0
  count = -1
  for i in train_img_inds:
    count += 1
    img = cv2.imread("/content/Chess Detection/images/" + imgs[i])
    img = cv2.resize(img , (500 , 281))
    whites , blacks , negs = img_to_prop[imgs[i]]
    min_len = min(len(whites) , len(blacks))
    if len(whites) + len(blacks) < 200:
      white_inds = list(range(len(whites)))
      black_inds = list(range(len(blacks)))
    else:
      if min_len == len(whites):
        white_inds = list(range(len(whites)))
        black_inds = np.random.choice(range(0 , len(blacks)) , (max(min_len , 1)) , replace = False)
      else:
        black_inds = list(range(len(blacks)))
        white_inds = np.random.choice(range(0 , len(whites)) , (max(min_len , 1)) , replace = False)
    neg_inds = np.random.choice(range(0 , len(negs)) , (max(min_len , 1)) , replace = False)
    tens = None
    coords = None
    for inds in white_inds:
      (xr , yr , wr , hr) , (xt , yt , xft , yft) = whites[inds]
      if coords is None:
        coords = torch.tensor([xt , yt , xft - xt , yft - yt])[None , ...]
      else:
        coords = torch.vstack([coords , torch.tensor([xt , yt , xft - xt , yft - yt])])
      temp = torch.from_numpy(cv2.resize(img[yr : yr + hr , xr : xr + wr] , (244 , 244)))[None , ...]
      if tens is None:
        tens = temp
      else:
        tens = torch.cat([tens , temp] , dim = 0)
    for inds in black_inds:
      (xr , yr , wr , hr) , (xt , yt , xft , yft) = blacks[inds]
      if coords is None:
        coords = torch.tensor([xt , yt , xft - xt , yft - yt])[None , ...]
      else:
        coords = torch.vstack([coords , torch.tensor([xt , yt , xft - xt , yft - yt])])
      temp = torch.from_numpy(cv2.resize(img[yr : yr + hr , xr : xr + wr] , (244 , 244)))[None , ...]
      if tens is None:
        tens = temp
      else:
        tens = torch.cat([tens , temp] , dim = 0)
    for inds in neg_inds:
      (xr , yr , wr , hr) , (xt , yt , xft , yft) = negs[inds]
      temp = torch.from_numpy(cv2.resize(img[yr : yr + hr , xr : xr + wr] , (244 , 244)))[None , ...]
      tens = torch.cat([tens , temp] , dim = 0)
    labels = torch.cat([torch.ones(len(white_inds)) , 
                        torch.ones(len(black_inds)) * 2 , 
                        torch.zeros(len(neg_inds))] , dim = 0).float()
    coords = coords.float()
    
    #training
    output_feat = model(tens.permute(0 , 3 , 1 , 2).float())
    cls_out = c_lay(output_feat)
    reg_out = r_lay(output_feat)
    loss_cls = loss_fn_cls(cls_out.squeeze() , labels.squeeze())
    loss_reg = loss_fn_reg(reg_out[:coords.shape[0] , ] , coords)

    loss = loss_cls + loss_reg

    epoch_loss += loss.cpu().detach().numpy()

    optimizer1.zero_grad()
    optimizer2.zero_grad()
    optimizer3.zero_grad()
    loss.backward()
    optimizer1.step()
    optimizer2.step()
    optimizer3.step()
    print(count  ,"image done")
  all_losses.append(epoch_loss)


In [ ]:
checkpoints = {"model_state" : model.state_dict() , 
               "optimizer1_state" : optimizer1.state_dict() , 
               "optimizer2_state" : optimizer2.state_dict() ,
               "optimizer3_state" : optimizer3.state_dict()}
torch.save(checkpoints , "five_epoch.pth.tar")

In [ ]:
!mv /content/five_epoch.pth.tar /content/drive/MyDrive/pytorch/practise/rcnn

In [ ]:
!mv /content/data.pkl /content/drive/MyDrive/pytorch/practise/rcnn

In [ ]:
all_whites = []
all_blacks = []
all_negs = []
model.eval()
with torch.no_grad():
  for i in train_img_inds:
    img = cv2.imread("/content/Chess Detection/images/" + imgs[i])
    img = cv2.resize(img , (500 , 281))
    whites , blacks , negs = img_to_prop[imgs[i]]
    for (xr , yr , wr , hr) , (xt , yt , xft , yft) in whites:
      cropped = cv2.resize(img[yr : yr + hr , xr : xr + wr] , (224 , 224))
      torch_img = torch.from_numpy(cropped)[None , ...]
      output = model(torch_img.permute(0 , 3 , 1 , 2).float())
      all_whites.append(output.squeeze().detach().numpy())
    for (xr , yr , wr , hr) , (xt , yt , xft , yft) in blacks:
      cropped = cv2.resize(img[yr : yr + hr , xr : xr + wr] , (224 , 224))
      torch_img = torch.from_numpy(cropped)[None , ...]
      output = model(torch_img.permute(0 , 3 , 1 , 2).float())
      all_blacks.append(output.squeeze().detach().numpy())
    num = min(len(negs) , max(len(whites) , len(blacks)))
    negs_inds = np.random.choice(range(0 , len(negs)) , num , replace = False)
    for j in negs_inds:
      (xr , yr , wr , hr) , (xt , yt , xft , yft) = negs[j]
      cropped = cv2.resize(img[yr : yr + hr , xr : xr + wr] , (224 , 224))
      torch_img = torch.from_numpy(cropped)[None , ...]
      output = model(torch_img.permute(0 , 3 , 1 , 2).float())
      all_negs.append(output.squeeze().detach().numpy())
    print(i , " img done")

In [ ]:
all_whites

In [ ]:
all_val_whites = []
all_val_blacks = []
all_val_negs = []
model.eval()
with torch.no_grad():
  for i in val_img_inds:
    img = cv2.imread("/content/Chess Detection/images/" + imgs[i])
    img = cv2.resize(img , (500 , 281))
    whites , blacks , negs = img_to_prop[imgs[i]]
    for (xr , yr , wr , hr) , (xt , yt , xft , yft) in whites:
      cropped = cv2.resize(img[yr : yr + hr , xr : xr + wr] , (224 , 224))
      torch_img = torch.from_numpy(cropped)[None , ...]
      output = model(torch_img.permute(0 , 3 , 1 , 2).float())
      all_val_whites.append(output.squeeze().detach().numpy())
    for (xr , yr , wr , hr) , (xt , yt , xft , yft) in blacks:
      cropped = cv2.resize(img[yr : yr + hr , xr : xr + wr] , (224 , 224))
      torch_img = torch.from_numpy(cropped)[None , ...]
      output = model(torch_img.permute(0 , 3 , 1 , 2).float())
      all_val_blacks.append(output.squeeze().detach().numpy())
    num = min(len(negs) , max(len(whites) , len(blacks)))
    negs_inds = np.random.choice(range(0 , len(negs)) , num , replace = False)
    for j in negs_inds:
      (xr , yr , wr , hr) , (xt , yt , xft , yft) = negs[j]
      cropped = cv2.resize(img[yr : yr + hr , xr : xr + wr] , (224 , 224))
      torch_img = torch.from_numpy(cropped)[None , ...]
      output = model(torch_img.permute(0 , 3 , 1 , 2).float())
      all_val_negs.append(output.squeeze().detach().numpy())
    print(i , " img done")

In [ ]:
clf_white = svm.SVC()
clf_black = svm.SVC()

In [ ]:
#training whites
nw = len(all_whites)
rest = np.random.choice(range(0 , max(nw , len(all_blacks + all_negs))) , nw , replace = False)
df_rest = np.vstack([all_blacks , all_negs])[rest]
white_df = pd.DataFrame(np.vstack([np.array(all_whites) , df_rest]))
targs = np.hstack([np.ones(nw) , np.zeros(nw)])
for i in range(5):
  clf_white.fit(white_df , targs)
  pred = clf_white.predict(white_df)
  print((pred == targs).sum() / targs.shape[0])

0.8276497695852535
0.8276497695852535
0.8276497695852535
0.8276497695852535
0.8276497695852535


In [ ]:
#training blacks
nw = len(all_blacks)
rest = np.random.choice(range(0 , max(nw , len(all_whites + all_negs))) , nw , replace = False)
df_rest = np.vstack([all_whites , all_negs])[rest]
black_df = pd.DataFrame(np.vstack([np.array(all_blacks) , df_rest]))
targs = np.hstack([np.ones(nw) , np.zeros(nw)])
for i in range(5):
  clf_black.fit(black_df , np.hstack([np.ones(nw) , np.zeros(nw)]))
  pred = clf_black.predict(black_df)
  print((pred == targs).sum() / targs.shape[0])

0.9310557094490612
0.9310557094490612
0.9310557094490612
0.9310557094490612
0.9310557094490612


In [ ]:
#testing whites
stack_whites = np.vstack([all_val_whites , all_val_blacks , all_val_negs])
real_targs = np.hstack([np.ones(len(all_val_whites)) , np.zeros(len(all_val_blacks) + len(all_val_negs))])
pred = clf_white.predict(pd.DataFrame(stack_whites))
(pred == real_targs).sum() / real_targs.shape[0]

0.7313624678663239

In [ ]:
#testing blacks
stack_blacks = np.vstack([all_val_blacks , all_val_whites , all_val_negs])
real_targs = np.hstack([np.ones(len(all_val_blacks)) , np.zeros(len(all_val_whites) + len(all_val_negs))])
pred = clf_black.predict(pd.DataFrame(stack_blacks))
(pred == real_targs).sum() / real_targs.shape[0]

0.9370179948586118

In [ ]:
#teating on training set for whites
nw = len(all_whites)
rest = np.random.choice(range(0 , max(nw , len(all_blacks + all_negs))) , nw , replace = False)
df_rest = np.vstack([all_blacks , all_negs])[rest]
white_df = pd.DataFrame(np.vstack([np.array(all_whites) , df_rest]))
pred = clf_white.predict(white_df) # , np.hstack([np.ones(nw) , np.zeros(nw)])
targs = np.hstack([np.ones(nw) , np.zeros(nw)])
(pred == targs).sum() / targs.shape[0]

0.8290322580645161

In [ ]:
#testing on training set for blacks
nw = len(all_blacks)
rest = np.random.choice(range(0 , max(nw , len(all_whites + all_negs))) , nw , replace = False)
df_rest = np.vstack([all_whites , all_negs])[rest]
black_df = pd.DataFrame(np.vstack([np.array(all_blacks) , df_rest]))
pred = clf_black.predict(black_df) # , np.hstack([np.ones(nw) , np.zeros(nw)])
targs = np.hstack([np.ones(nw) , np.zeros(nw)])
(pred == targs).sum() / targs.shape[0]

0.9269005847953217